<a href="https://colab.research.google.com/github/subhadrastudent/AI-Healthcare/blob/main/AICTE_Week2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ----------------------------------
# Step 1: Download & Explore Dataset
# ----------------------------------
import kagglehub
path = kagglehub.dataset_download("elmadafri/the-wildfire-dataset")
print("Path to dataset files:", path)

# ----------------------------------
# Step 2: Import Required Libraries
# ----------------------------------
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

# ----------------------------------
# Step 3: GPU Check
# ----------------------------------
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if physical_devices:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
    print("✅ GPU Available")
else:
    print("⚠️ GPU Unavailable")

# ----------------------------------
# Step 4: Define Data Paths
# ----------------------------------
train_dir = '/kaggle/input/the-wildfire-dataset/the_wildfire_dataset_2n_version/train'
val_dir = '/kaggle/input/the-wildfire-dataset/the_wildfire_dataset_2n_version/val'
test_dir = '/kaggle/input/the-wildfire-dataset/the_wildfire_dataset_2n_version/test'

classes = os.listdir(train_dir)
print(f"Classes: {classes}")

# ----------------------------------
# Step 5: Data Augmentation & Generator
# ----------------------------------
img_size = (150, 150)
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=25,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

val_test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir, target_size=img_size, batch_size=batch_size, class_mode='binary', shuffle=True)

val_generator = val_test_datagen.flow_from_directory(
    val_dir, target_size=img_size, batch_size=batch_size, class_mode='binary', shuffle=False)

test_generator = val_test_datagen.flow_from_directory(
    test_dir, target_size=img_size, batch_size=batch_size, class_mode='binary', shuffle=False)

# ----------------------------------
# Step 6: Model Architecture
# ----------------------------------
model = Sequential([
    Input(shape=(150, 150, 3)),

    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    BatchNormalization(),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.4),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

# ----------------------------------
# Step 7: Callbacks
# ----------------------------------
checkpoint_cb = ModelCheckpoint("best_fire_model.h5", save_best_only=True, monitor='val_loss')
earlystop_cb = EarlyStopping(patience=5, restore_best_weights=True)

# ----------------------------------
# Step 8: Train Model
# ----------------------------------
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=20,
    callbacks=[checkpoint_cb, earlystop_cb]
)

# ----------------------------------
# Step 9: Evaluate Model
# ----------------------------------
loss, acc = model.evaluate(test_generator)
print(f"Test Accuracy: {acc:.2f}")

# Confusion Matrix
y_pred = model.predict(test_generator)
y_pred_labels = (y_pred > 0.5).astype(int).flatten()
y_true = test_generator.classes

print("\nClassification Report:\n", classification_report(y_true, y_pred_labels, target_names=classes))

cm = confusion_matrix(y_true, y_pred_labels)
sns.heatmap(cm, annot=True, fmt='d', xticklabels=classes, yticklabels=classes, cmap='Blues')
plt.title("Confusion Matrix")
plt.show()

# ----------------------------------
# Step 10: Fire Alert System (Sample)
# ----------------------------------
def predict_fire(image_path):
    from tensorflow.keras.preprocessing import image
    img = image.load_img(image_path, target_size=img_size)
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    prediction = model.predict(img_array)[0][0]
    if prediction > 0.5:
        print("🔥 Alert: Forest Fire Detected!")
    else:
        print("🌲 Safe: No Fire Detected.")

# Example usage:
# predict_fire('/path/to/test/image.jpg')


Path to dataset files: /kaggle/input/the-wildfire-dataset
✅ GPU Available
Classes: ['nofire', 'fire']
Found 1887 images belonging to 2 classes.
Found 402 images belonging to 2 classes.
Found 410 images belonging to 2 classes.


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 148, 148, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 72, 72, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 36, 36, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 36, 36, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 34, 34, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 17, 17, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 36992)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │     9,470,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,563,969 (36.48 MB)

 Trainable params: 9,563,841 (36.48 MB)

...

/usr/local/lib/python3.11/dist-packages/PIL/Image.py:3402: DecompressionBombWarning: Image size (89747104 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


49/59 ━━━━━━━━━━━━━━━━━━━━ 1:06 7s/step - accuracy: 0.5588 - loss: 2.9505

/usr/local/lib/python3.11/dist-packages/PIL/Image.py:3402: DecompressionBombWarning: Image size (104688771 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 7s/step - accuracy: 0.5665 - loss: 2.6827

59/59 ━━━━━━━━━━━━━━━━━━━━ 504s 8s/step - accuracy: 0.5673 - loss: 2.6598 - val_accuracy: 0.5373 - val_loss: 0.6912
Epoch 2/20
59/59 ━━━━━━━━━━━━━━━━━━━━ 461s 8s/step - accuracy: 0.7082 - loss: 0.5457 - val_accuracy: 0.3905 - val_loss: 0.7183
Epoch 3/20
59/59 ━━━━━━━━━━━━━━━━━━━━ 479s 8s/step - accuracy: 0.7399 - loss: 0.5094 - val_accuracy: 0.3905 - val_loss: 0.8092
Epoch 4/20
59/59 ━━━━━━━━━━━━━━━━━━━━ 526s 9s/step - accuracy: 0.7298 - loss: 0.5219 - val_accuracy: 0.4851 - val_loss: 0.7482
Epoch 5/20
 8/59 ━━━━━━━━━━━━━━━━━━━━ 5:21 6s/step - accuracy: 0.8079 - loss: 0.4437